In [1]:
import time

import jax.numpy as jnp
import numpy as np

from vehicle_model_jax import compute_jacobian_jax, get_default_params

In [2]:
p = get_default_params()
p_tuple = tuple(p.values())

In [6]:
x0 = jnp.zeros(10)
u = jnp.array([0.0, 0.0])

J_jax = compute_jacobian_jax(x0, p_tuple, u)
print(J_jax)

[[-5.0499995e+03  0.0000000e+00  4.9999996e+04  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -3.3666667e+03  0.0000000e+00  3.3333336e+04
   0.0000000e+00  0.0000000e+00  0.0000000e+00 -9.0643281e+03
   0.0000000e+00  3.5087722e+03]
 [-1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -1.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00 -1.0526316e+00
   0.0000000e+00  1.0526316e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e